# Grayscale Image Compression

In [1]:
import numpy as np
import cv2
import sys

In [2]:
def readImg(filename):
    img = cv2.imread(filename)

    if img is None:
        print("Failed to load image file.")
        sys.exit(1)
        
    imgArray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    height, width = imgArray.shape
    print("width: " + str(width))
    print("height: " + str(height))
    print("dtype : " + str(img.dtype))

    cv2.imshow('Original Gray Image',imgArray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()  
    
    return compression(imgArray)

In [3]:
def trim(img):
    """This function makes sure that the image have an even number of pixels, else the haar-transform will not work.
       If the rows/columns of the image is odd then cut off the last row/column and save the image."""
        
    height, width = img.shape[0], img.shape[1]
    
    if height % 2 !=0:
        img = img[:-1]
        print ('Image was uneven (width-wise) and has been trimmed!')

    if width %2 !=0:
        imgT = img.transpose()
        imgT = imgT[:-1]
        img = imgT.transpose()
        print ('Image was uneven (height-wise) and has been trimmed!')

    print('Image size is {}x{}'.format(img.shape[0],img.shape[1]))

    cv2.imwrite('Trimmed_img.png', img)
    img=cv2.imread('Trimmed_img.png',0)
    
    return img, img.shape[0], img.shape[1]
    

In [4]:
def haar(sz):
    """This function creates the Haar-matrix."""

    out = np.zeros((sz, sz))

    j=0
    for i in range(int(sz/2)):
       
        out[i,j]=np.sqrt(2)/2
        out[i,j+1]=np.sqrt(2)/2
        j+=2
    
    j=0
    for i in range(int(sz/2), sz):
       
        out[i,j]=-np.sqrt(2)/2
        out[i,j+1]=np.sqrt(2)/2
        j+=2

    return out

If you want to see how the haar-matrix looks like, write size n of the matrix (nxn) you want.

In [5]:
#haar(10)

In [6]:
def compression(imgArray):
    """This function trims and compresses an image using dot-products and the Haar-matrix"""
    img, m_size, n_size = trim(imgArray)
    w_m=haar(m_size)
    w_t=haar(n_size).T

    """Here is the first dotproduct"""    
    comp1=np.dot(w_m,img)
    cv2.imwrite('first_dotproduct.png', comp1) 
    comp1 = comp1.astype(np.uint8)       #change dtype to uint8 from float 64
    comp1=cv2.imread('first_dotproduct.png',0)
    cv2.imshow('first_dotproduct.png',comp1)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    
    
    """Here is the second dotproduct"""
    comp2=np.dot(img,w_t)
    cv2.imwrite('second_dotproduct.png', comp2) 
    comp2 = comp2.astype(np.uint8)       #change dtype to uint8 from float 64
    comp1=cv2.imread('second_dotproduct.png',0)
    cv2.imshow('second_dotproduct.png',comp2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    """Finally both dotproducts together"""
    comp=np.dot(np.dot(w_m,img),w_t)
    cv2.imwrite('both_dotproducts.png', comp) 
    comp = comp1.astype(np.uint8)       #change dtype to uint8 from float 64
    comp=cv2.imread('both_dotproducts.png',0)
    cv2.imshow('both_dotproducts.png',comp)
    cv2.waitKey(0)
    cv2.destroyAllWindows()  
    

    return comp   

In [7]:
def topLeft(comp_img):
    """This function cuts the top-left fourth of the image"""
    imgz = np.shape(comp_img)
    img = np.array(comp_img[0:imgz[0]//2, 0:imgz[1]//2])
    cv2.imwrite('top_left.png', img)       
    
    img = img.astype(np.uint8) #change dtype to uint8 from float 64
    img=cv2.imread('top_left.png',0)
    cv2.imshow('top_left.png',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()  
    
    return img

In [8]:
def multiComp(filename, times):
    """This function compresses a image as many times as the you tell it to."""
    #TODO: add exceptio for when size is to small
    compd = readImg(filename)
    
    if times == 1:
        return topLeft(compd)
    else:
        while times > 1:
            compd = topLeft(compd)
            compd = compression(compd)
            times -= 1
        
    return topLeft(compd)

In [11]:
multiComp('kvinna.png', 2)

width: 478
height: 600
dtype : uint8
Image size is 600x478
Image was uneven (height-wise) and has been trimmed!
Image size is 300x238


array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)